In [1]:
import numpy as np
import torch
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
import os

In [2]:

import sys
import numpy as np
# add gensim source code fro this path
sys.path.insert(0, "../../../")
sys.path.insert(0, "../../gensim/")

# import modules & set up logging
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
                
                

## Import the training data from file

In [3]:
from smart_open import smart_open
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in smart_open(os.path.join(self.dirname, fname), 'r'):
                yield line.split()

In [6]:
# point to sentences to be learned file here
my_sentences_fn = '../..//ULL/data/Practical2/wa/english_full/'
sentences = MySentences(my_sentences_fn) # a memory-friendly iterator
print(list(sentences))

[['2', '.'], ['2', '.'], ['oh', ',', 'oh', '!'], ['oh', ',', 'oh', '!'], ['oh', ',', 'oh', '!'], ['let', 'us', 'remember', ',', 'Mr.', 'Speaker', ',', 'that', 'these', 'segments', 'of', 'our', 'society', 'form', 'the', 'backbone', 'of', 'our', 'economy', '.'], ['hear', ',', 'hear', '!'], ['hear', ',', 'hear', '!'], ['Mr.', 'Speaker', ',', 'my', 'question', 'is', 'directed', 'to', 'the', 'Minister', 'of', 'Transport', '.'], ['both', 'have', 'many', 'years', 'experience', 'in', 'the', 'manufacture', 'and', 'distribution', 'of', 'forest', 'products', '.'], ['1', '.'], ['another', 'point', 'I', 'should', 'like', 'to', 'discuss', 'is', 'the', 'right', 'of', 'the', 'supplier', 'to', 'choose', 'his', 'customers', 'as', 'he', 'sees', 'fit', '.'], ['i', 'am', 'getting', 'sick', 'and', 'tired', 'of', 'ministers', 'who', 'seem', 'to', 'have', 'some', 'hang', '-', 'up', 'with', 'regard', 'to', 'the', 'collective', 'bargaining', 'process', '.'], ['there', 'is', 'just', 'one', 'specific', 'item', 'm

In [7]:

# corpus = [
#     'he is a king',
#     'she is a queen',
#     'he is a man',
#     'she is a woman',
#     'warsaw is poland capital',
#     'berlin is germany capital',
#     'paris is france capital',
# ]


# def tokenize_corpus(corpus):
#     tokens = [x.split() for x in corpus]
#     return tokens

# tokenized_corpus = tokenize_corpus(corpus)


vocabulary = []
for sentence in sentences:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}


vocabulary_size = len(vocabulary)

In [8]:
for i in vocabulary:
    if i == 'he':
        print(i)

he


In [9]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in sentences:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array



In [10]:

def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [11]:
print(np.shape(W2))

NameError: name 'W2' is not defined

In [16]:
import time
start_time = time.time()

end_time=time.time()
print("Took %.2f"%(end_time-start_time))

Took 0.00


In [17]:
import time
start_time = time.time()

embedding_dims = 5
# D: Latent variables / embedding dimensions
# V
# W1: shape(Zn x Nn)
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 101
learning_rate = 0.001


for epo in range(num_epochs):
    loss_val = 0
    # Compute loss for each training pair after predicting it with the NN
    for data, target in idx_pairs:
        # input layer: hot vectors?
        x = Variable(get_input_layer(data)).float()
        #print(data,target,  x)
        y_true = Variable(torch.from_numpy(np.array([target])).long())
        
        # this selects the embedding corresponding to x hot vector
        # z1: size(1xZ) column vec
        z1 = torch.matmul(W1, x)
        
        # multiply selected embedding, z1 with contexts matrix
        z2 = torch.matmul(W2, z1)
        
        # pass z2 through softmax to get probabilities
        # and compute log
        log_softmax = F.log_softmax(z2, dim=0)
        
        # subtract model predictions from true contexts
        #print(log_softmax.view(1,-1))
        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        #print("loss",loss.data[0])
        loss_val += loss.data[0]
        # Run backprop
        loss.backward()
        # Compute gradient of the parameters matrix W1, evaluate at data
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print('Loss at epo %s: %s/%s'%(epo, loss_val, len(idx_pairs)))
        

end_time=time.time()
print("Took %.2f"%(end_time-start_time))

/Users/efiathieniti/miniconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:38: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Loss at epo 0: tensor(1.00000e+05 *
       2.3901)/25398
Loss at epo 10: tensor(1.00000e+05 *
       2.0631)/25398
Loss at epo 20: tensor(1.00000e+05 *
       1.9564)/25398
Loss at epo 30: tensor(1.00000e+05 *
       1.8509)/25398
Loss at epo 40: tensor(1.00000e+05 *
       1.7717)/25398
Loss at epo 50: tensor(1.00000e+05 *
       1.7147)/25398
Loss at epo 60: tensor(1.00000e+05 *
       1.6715)/25398
Loss at epo 70: tensor(1.00000e+05 *
       1.6376)/25398
Loss at epo 80: tensor(1.00000e+05 *
       1.6103)/25398
Loss at epo 90: tensor(1.00000e+05 *
       1.5881)/25398
Loss at epo 100: tensor(1.00000e+05 *
       1.5698)/25398
Took 982.94


In [18]:
print(vocabulary_size)
# Embeddings are the W1

print(vocabulary[1])

1732
.


In [19]:
W1.detach().numpy()[:,1]

array([ 0.17221825, -0.19177108,  0.54171652,  0.4284915 , -0.06143791], dtype=float32)

## Save the embeddings learned as gensim model

In [20]:
import csv
embeddings_file='embeddings_new.txt'
with open(embeddings_file, 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter = ' ')
    writer.writerow([vocabulary_size, embedding_dims])
    for i in range(vocabulary_size):    
        line= [vocabulary[i]] + list(W1.detach().numpy()[:,i])
        writer.writerow(line)
    

In [21]:
embeddings_file='embeddings_new.txt'


## Load the embeddings learned 

In [22]:

from gensim.models import KeyedVectors
#word_vectors = KeyedVectors.load_word2vec_format('../../../ULL/data/bow2.words.bz2', binary=False)  # C text format
word_vectors = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)  # C binary format

2018-04-29 01:07:03,528 : INFO : loading projection weights from embeddings_new.txt
2018-04-29 01:07:03,550 : INFO : loaded (1732, 5) matrix from embeddings_new.txt


In [23]:
for i in vocabulary:
    print(i)

2
.
oh
,
!
let
us
remember
Mr.
Speaker
that
these
segments
of
our
society
form
the
backbone
economy
hear
my
question
is
directed
to
Minister
Transport
both
have
many
years
experience
in
manufacture
and
distribution
forest
products
1
another
point
I
should
like
discuss
right
supplier
choose
his
customers
as
he
sees
fit
i
am
getting
sick
tired
ministers
who
seem
some
hang
-
up
with
regard
collective
bargaining
process
there
just
one
specific
item
more
would
comment
upon
then
will
sit
down
order
please
3
can
refer
him
no
better
authority
on
subject
than
hon.
member
for
Don
Valley
not
myself
it
quite
understandable
addition
could
become
a
serious
threat
Confederation
national
unity
view
this
we
deemed
inadvisable
attend
meeting
so
informed
COJO
minister
suggested
greater
building
program
might
lead
inflationary
pressure
disagree
argument
advanced
by
lot
be
said
sides
indicated
Leader
Opposition
hope
make
an
announcement
November
does
bring
anybody
closer
rehabilitation
isolate
from
public


In [38]:
print(word_vectors.similarity('disturbed', 'pervasive'))


0.64288


In [25]:
word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])
[('queen', 0.50882536), ...]

word_vectors.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
[('queen', 0.71382287), ...]

word_vectors.doesnt_match("breakfast cereal dinner lunch".split())
'cereal'



2018-04-29 01:07:11,826 : INFO : precomputing L2-norms of word weight vectors


KeyError: "word 'woman' not in vocabulary"

In [26]:
word_vectors.doesnt_match("breakfast cereal dinner lunch".split())
'cereal'


2018-04-29 01:07:48,277 : WARNING : vectors for words {'dinner', 'cereal', 'lunch', 'breakfast'} are not present in the model, ignoring these words


ValueError: cannot select a word from an empty list

In [27]:
word_vectors.similarity('woman', 'man')


KeyError: "word 'woman' not in vocabulary"